In [3]:
import urllib.request
import urllib.error
import requests 
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import csv
import pandas as pd
accepted_scheme = ['http://', 'https://', 'ftp']


In [54]:
def read_url():
    """
    reads the url from url.txt
    returns : string
    """
    f1 = open("inputs/url.txt", "r")
    url = f1.read()
    return url

In [117]:
url = "https://www.ndtv.com/india-news/coronavirus-west-bengal-pins-covid-19-testing-delays-on-defective-kits-from-top-medical-body-icmr-2214619?pfrom=home-bigstory"

In [56]:
def check(domain):
    """
    args: url string
    return:  True; if the URL is not present in the database
             the 1st element, which is the justification for the site being in the list; otherwise
    Used to check if the given 'domain' argument is present in the database
    """
    with open('scraping/sources.csv', 'r') as csv_data:
        fields = csv.reader(csv_data,delimiter="\t")
        # Reads the column titles in the field
        for row in fields:
            # Reads the value in each row, into a list for columns
            if domain == row[0]:
#                 print(row[1])
                pr = str(row[1])
                if(row[2]):
                    pr += "   "+ row[2] 
                if(row[3]):
                    pr += "   type  "+ row[3]
                if(row[4]):
                    pr += "   problem: "+ row[4]
                print(str(url)+" reported as "+ pr +'\n')
                display_write(str(url)+" reported as "+ pr +'\n')
                # Compares with the 0th element in the list 'row', which are the domain URLs
                return row[1]
    return 1
                
    

In [57]:
def display_write(str):
    """
    args: string
    writes string to display.txt file which is to be dispalyed to user
    """
    str = "\n"+str+"\n"
    file = open("display.txt",'a')
    file.write(str)
    

In [93]:

def getStrippedLink(link):
    """
    args: url string
    Used to generate a string with only the domain
    """
    parse = urlparse(link)
    # Parse the given link as a URL
    stripped_link = parse[1]
    # Get the domain specific string from the list 'parse'
    if 'www.' in link or 'http://' in link or 'https://' in link:
        # Check for the given strings in the link and remove them 
        stripped_link = stripped_link.replace('https://','').replace('http://','').replace('www.','')
    #print(stripped_link)
    return stripped_link


def openURL(link):
    """
    agrs: url string
    Create a Request object for the given link, with a user-agent specified 
    """
    web_link = urllib.request.Request(link, data=None, headers={'User-Agent': 'Mozilla/5.0'})
    try:
        handle = urllib.request.urlopen(web_link)
        # Attempt to connect to the URL and store the HTML GET response in 'handle' variable
    except urllib.error.HTTPError:
        print("Page unavailable")
        exit()
    return handle


def fileTitleWrite(soup, link,filename):
    """
    agrs: soup object, url string
    Write the domain and title to the 'title.txt' file
    """ 
    fo = open(filename, "w")
    tit = soup.title.text
    fo.write(link + "\n")
    tit = soup.title.text
    fo.write(tit)
    fo.close()
    print(tit)
    return tit



def fileLinkWrite(soup, parse):
    '''
    args: soup object, parse string
    Write the hyperlinked URLs to the 'links.txt' file
    '''
    fo = open("links.txt", "w")
    for link in soup.findAll('a'):
        # Find all the 'a' tags
        if 0 <= str(link.get('href')).find(str(parse[1])):
            # Find the tags with 'href' and which do not match to given link's domain 
            continue
        else:
            for s in accepted_scheme:
                if 0 <= str(link.get('href')).find(s):
                    # If accepted connection type, write the hyperlink to the file
                    fo.write(str(link.get('href')) + "\n")
    fo.close()

def fileArticleWrite(soup, link,tit,filename):
    # Write the domain and title to the 'title.txt' file
    p_tags = soup.find_all('p')
    # Get the text from each of the “p” tags and strip surrounding whitespace.
    p_tags_text = [tag.get_text().strip() for tag in p_tags]
    sentence_list = [sentence for sentence in p_tags_text if not '\n' in sentence]
    sentence_list = [sentence for sentence in sentence_list if '.' in sentence]
    article = tit+ " "
    article += ' '.join(sentence_list)
    fo = open(filename, "w")
    fo.write(article)
    fo.close()
    
def parse(link):
    """
    args: url string
    Runs the entire script
    """
    # Runs the entire script
    stripped_link = getStrippedLink(link)
    page = requests.get(link).text
    #Parse the pase into res
    soup = BeautifulSoup(page)
    #Use BeautifulSoup to create a nested data structure out of the HTML file of the website
    tit = fileTitleWrite(soup, stripped_link,"title.txt")
    urls = pred(link,tit)
    print(urls)
    fileLinkWrite(soup, urlparse(link))
    fileArticleWrite(soup, link,tit,"article.txt")

In [94]:

def checkDomain():
    """
    Check the URL of the given website, by invoking the check function 
    """ 
    fo = open('title.txt')
    link = fo.readline().rstrip("\n")
    fo.close()
    # Reading from the 'title.txt' file for the link of the site
    ret = check(link)
    # Store the return value of the check function
    if ret==1:
        return True
    else:
        reasons = check(link)
        return False


def checkLinks():
    """
    Check the domain of all the hyperlinks in the given URL
    """
    fo = open('links.txt')
    for line in fo:
        link = getStrippedLink(line)
        if check(link):
            return False
    return True


def result(link):
    """ 
    args: url string
    Main function for invoking all the respective scripts
    """
    
    link = str(link)
    parse(link)
    results = ""
    if checkDomain():
        r = checkLinks()
#         print(r)
        if r:
            results = "The site is not a fake news site"
            # If the response is True, none of the hyperlinked URLs are present in the database
        else:
            results = "The site has sources from a fake site"
            # If the response is False, one or more hyperlinked URLs are present in the database
    else:
        results ="The site is a fake news site"
    display_write(str(results))
    print(results)


In [105]:
def get_text_from_url(url):
    end = url.find('.html')
    print(end)
#     url[start: end]

In [106]:
get_text_from_url(url)

70


In [119]:
keywords = url.replace(".","").replace("?","").replace(".html"," ").replace("https://"," ").replace("-"," ").replace("www"," ").replace("hhtp://"," ").replace(".com"," ").replace(".in"," ").replace("?","").replace("/","")

In [120]:
print(keywords)

  ndtvcomindia newscoronavirus west bengal pins covid 19 testing delays on defective kits from top medical body icmr 2214619pfrom=home bigstory


In [99]:
def pred(link,title):
    """ 
    args: url string, title
    returns: related urls list 
    searches related urls from trusted sources and writes down their article body into "articles+soruceno.txt"
    """
    urls = find_related_urls(title)
    sourceno = 0
    for link in urls:
        page = requests.get(link).text
        #Parse the pase into res
        soup = BeautifulSoup(page)
        titlefile = "title"+str(sourceno)+".txt"
        articlefile = "article"+str(sourceno)+".txt"
        stripped_link = getStrippedLink(link)
        #Use BeautifulSoup to create a nested data structure out of the HTML file of the website
        fileArticleWrite(soup, link,title,articlefile)
        sourceno += 1
    return urls



In [135]:

def find_related_urls(title):
    """
    biggest innovation!
    args: title of article
    returns: links of  most related articles from trusted sources
    """
    try: 
        from googlesearch import search 
    except ImportError:  
        print("No module named 'google' found") 
    
    print(title)
    related_urls = []
    # to search 
    query1 = "ndtv: "+ title
    query2 = "timesofindia: "+title
    query3 = "hindustantimes: " + title
    print("related urls extracted from trusted sources")
    for q in search(query1, tld="com", num=10, stop=1, pause=2): 
        print(q)
        related_urls.append(q)
    for r in search(query2, tld="co.in", num=10, stop=1, pause=2): 
        print(r)
        related_urls.append(r)
    for s in search(query3, tld="com", num=10, stop=1, pause=2): 
        print(s)
        related_urls.append(s)
#     print(related_urls)
    return related_urls
        

In [136]:
find_related_urls("Coronavirus updates")

Coronavirus updates
related urls extracted from trusted sources
https://khabar.ndtv.com/news/india/india-coronavirus-lockdown-updates-41-deaths-1409-new-cases-were-reported-in-last-24-hours-2216625
https://timesofindia.indiatimes.com/india/coronavirus-cases-in-india-live-updates-total-number-of-corona-cases-in-india-rises-to-19984-with-640-deaths/liveblog/75283503.cms
https://www.hindustantimes.com/topic/coronavirus


['https://khabar.ndtv.com/news/india/india-coronavirus-lockdown-updates-41-deaths-1409-new-cases-were-reported-in-last-24-hours-2216625',
 'https://timesofindia.indiatimes.com/india/coronavirus-cases-in-india-live-updates-total-number-of-corona-cases-in-india-rises-to-19984-with-640-deaths/liveblog/75283503.cms',
 'https://www.hindustantimes.com/topic/coronavirus']

In [137]:
result(url)

Coronavirus: West Bengal Pins COVID-19 Testing Delays On Defective Kits From Top Medical Body ICMR
Coronavirus: West Bengal Pins COVID-19 Testing Delays On Defective Kits From Top Medical Body ICMR
related urls extracted from trusted sources
https://www.ndtv.com/india-news/coronavirus-west-bengal-pins-covid-19-testing-delays-on-defective-kits-from-top-medical-body-icmr-2214619
https://www.ndtv.com/india-news/coronavirus-west-bengal-pins-covid-19-testing-delays-on-defective-kits-from-top-medical-body-icmr-2214619
https://www.ndtv.com/india-news/coronavirus-west-bengal-pins-covid-19-testing-delays-on-defective-kits-from-top-medical-body-icmr-2214619
['https://www.ndtv.com/india-news/coronavirus-west-bengal-pins-covid-19-testing-delays-on-defective-kits-from-top-medical-body-icmr-2214619', 'https://www.ndtv.com/india-news/coronavirus-west-bengal-pins-covid-19-testing-delays-on-defective-kits-from-top-medical-body-icmr-2214619', 'https://www.ndtv.com/india-news/coronavirus-west-bengal-pins

In [102]:
from bs4 import BeautifulSoup
import requests


def extract(url):
    #r = requests.get('http://www.aurionpro.com/')
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html")

    title = soup.title.string
    print('TITLE IS :', title)

    meta = soup.find_all('meta')

    for tag in meta:
        if 'name' in tag.attrs.keys() and tag.attrs['name'].strip().lower() in ['description', 'keywords']:
            print('NAME    :',tag.attrs['name'].lower())
            print('CONTENT :',tag.attrs['content'])



In [103]:
extract(url)

TITLE IS : Economic Chaos Fuels Hunger and Strongmen - The New York Times
NAME    : description
CONTENT : The E.U. accused Poland and Hungary of undermining democracy — and gave them billions in aid. Iran may be executing prisoners who feared infection.
